In [329]:
pip install streamlit

In [331]:
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import streamlit as st


In [332]:
# Загрузка данных из файла
def load_data_from_file(file_path):
  try:
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    return data
  except FileNotFoundError:
      print(f"Файл '{file_path}' не найден.")
      return None

# Загрузка данных из yaho finance
def load_data_from_yf(ticker, timeframe='1d'):
  df = yf.download(ticker, interval=timeframe)
  df = df.reset_index()
  if isinstance(df.columns, pd.MultiIndex):
      df.columns = df.columns.droplevel(level=1)
  df['Date'] = pd.to_datetime(df['Date'])
  df.set_index('Date', inplace=True)
  return df

In [333]:
# Индикатор RSI
def rsi(close_prices, n=14):
  delta = close_prices.diff().dropna()
  up = delta.clip(lower=0)
  down = -delta.clip(upper=0)
  rs = up.ewm(span=n, adjust=False).mean() / down.ewm(span=n, adjust=False).mean()
  return 100 - (100 / (1 + rs))

In [334]:
# Колонки для нормализации
columns_to_scale = ['Open', 'High', 'Low', 'Close', 'Volume', 'Mean', 'Value']
# Колонки, которые не нужно нормализовать
columns_not_to_scale = ['SMA', 'RSI']

# Предобработка данных
def preprocess_data(data, window_size=30):
  # Добавление новых признаков
  data['Mean'] = (data['Open']+data['High']+data['Low']+data['Close']) / 4 # Средняя цена
  data['Value'] = data['Volume'] * data['Mean'] # Объём в деньгах
  data['SMA'] = data['Close'].rolling(window_size).mean()  # Скользящая средняя
  data['RSI'] = rsi(data['Close'], n=window_size)  # Индекс относительной силы

  # Удаляем строки с NaN-значениями
  data.dropna(inplace=True)

  # Нормализация данных только к выбранным колонкам
  scaler = MinMaxScaler(feature_range=(0, 1))
  scaled_data = scaler.fit_transform(data[columns_to_scale])

  # Преобразуем результат обратно в DataFrame
  scaled_df = pd.DataFrame(scaled_data, columns=columns_to_scale)

  # Добавляем колонки, которые не нужно было нормализовать
  scaled_df[columns_not_to_scale] = data[columns_not_to_scale].reset_index(drop=True)

  # Преобразуем DataFrame в массив NumPy для удобства работы
  scaled_data = scaled_df.values

  # Создание временных окон
  X, y = [], []
  for i in range(window_size, len(scaled_data)):
      X.append(scaled_data[i-window_size:i, :])
      y.append(scaled_data[i, 0]) #Целевая переменная
  X, y = np.array(X), np.array(y)

  # Разделение на обучающую и тестовую выборки
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
  return X_train, X_test, y_train, y_test, scaler

In [335]:
def create_lstm_model(input_shape):
  model = Sequential()
  model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
  model.add(Dropout(0.2))
  model.add(LSTM(50, return_sequences=False))
  model.add(Dropout(0.2))
  model.add(Dense(25))
  model.add(Dense(1))  # Прогнозирование цены
  model.compile(optimizer='adam', loss='mean_squared_error')
  return model

In [336]:
def create_cnn_lstm_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(25))
    model.add(Dense(1))  # Прогнозирование цены
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [337]:
def train_model(model, X_train, y_train, epochs=50, batch_size=32):
  history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, verbose=1)
  return history

In [352]:
def test_model(model, X_test, y_test):
    # Получаем прогнозы
    predictions = model.predict(X_test)
    # Создаем массив с 8 столбцами, где первый столбец — прогнозы, остальные — нули
    #predictions_reshaped = np.zeros((predictions.shape[0], scaler.n_features_in_))
    #predictions_reshaped[:, 0] = predictions.flatten()
    # Применяем обратное преобразование
    #predictions = scaler.inverse_transform(predictions_reshaped)[:, 0]
    # Аналогично для y_test
    #y_test_reshaped = np.zeros((y_test.shape[0], scaler.n_features_in_))
    #y_test_reshaped[:, 0] = y_test.flatten()
    #y_test = scaler.inverse_transform(y_test_reshaped)[:, 0]
    return predictions, y_test

In [339]:
def calculate_metrics(predictions, y_test):
  # метрика: точность направления движения цены
  pred_sign = np.sign(predictions[1:] - predictions[:-1])
  test_sign = np.sign(y_test[1:] - y_test[:-1])
  direction_accuracy = accuracy_score(test_sign, pred_sign)
  rmse = np.mean((predictions - y_test) ** 2) ** 0.5
  mae = np.mean(np.abs(predictions - y_test))
  average = y_test.mean()
  rmse2avg = rmse / average
  mae2avg = mae / average
  return direction_accuracy, rmse, mae, rmse2avg, mae2avg

In [340]:
# Загрузка данных
#file_path = 'stock_data.csv'  # Замените на путь к вашему файлу
#data = load_data_from_file(file_path)
data = load_data_from_yf("AAPL")

[*********************100%***********************]  1 of 1 completed


In [341]:
# Предобработка данных
X_train, X_test, y_train, y_test, scaler = preprocess_data(data)

In [342]:
# Создание и обучение модели
model = create_cnn_lstm_model((X_train.shape[1], X_train.shape[2]))
history = train_model(model, X_train, y_train, 10)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 21ms/step - loss: 0.0060 - val_loss: 1.8291e-04
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 6.5949e-05 - val_loss: 5.9810e-04
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 2.9363e-05 - val_loss: 0.0010
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 2.2707e-05 - val_loss: 6.5937e-04
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - loss: 1.8849e-05 - val_loss: 9.5077e-04
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - loss: 1.5660e-05 - val_loss: 0.0013
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - loss: 1.3692e-05 - val_loss: 5.8678e-04
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - loss: 1.6531e-05 - val_loss: 4.5637e-04
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - loss: 1.1982e-05 - val_loss: 8.4311e-04
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 1.2265e-05 - val_loss: 7.0676e-04


In [343]:
# Тестирование модели
predictions, y_test = test_model(model, X_test, y_test, scaler)

# Расчет метрик
direction_accuracy, rmse, mae, rmse2avg, mae2avg  = calculate_metrics(predictions, y_test)
print(f"Точность направления движения цены: {direction_accuracy:.4f}")
print(f"Средняя абсолютная ошибка цены (MAE): {mae:.4f}")
print(f"Средняя квадратичная ошибка цены (RMSE): {rmse:.4f}")
print(f"Относительная средняя абсолютная ошибка цены (MAE): {mae2avg:.4f}")
print(f"Относительная средняя квадратичная ошибка цены (RMSE): {rmse2avg:.4f}")

70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
Точность направления движения цены: 0.5038
Средняя абсолютная ошибка цены (MAE): 0.4059
Средняя квадратичная ошибка цены (RMSE): 0.4807
Относительная средняя абсолютная ошибка цены (MAE): 0.9829
Относительная средняя квадратичная ошибка цены (RMSE): 1.1641


In [344]:
# Сохранение метрик в файл
metrics = {
    'Модель': 'LSTM',
    'Точность направления': direction_accuracy,
    'Средняя абсолютная ошибка': mae,
    'Средняя квадратичная ошибка': rmse
}
metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv('metrics.csv', index=False)

In [349]:
# Создание дашборда с использованием Streamlit
def create_dashboard(data, predictions, y_test):
    st.title('Дашборд торговой стратегии')
    st.write("### График прогнозов и реальных цен")
    plt.figure(figsize=(10, 6))
    plt.plot(y_test, label='Реальные цены')
    plt.plot(predictions, label='Прогнозы')
    plt.legend()
    st.pyplot(plt)

    st.write("### Метрики модели")
    metrics_df = pd.read_csv('metrics.csv')
    st.table(metrics_df)

In [350]:
create_dashboard(data, predictions, y_test)

2025-02-22 17:16:15.063 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 17:16:15.064 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 17:16:15.065 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 17:16:15.066 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 17:16:15.067 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 17:16:15.068 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 17:16:15.087 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 17:16:15.574 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [353]:
def simulate_trading(model, X_test, y_test, initial_capital=10000, commission=0.001):
    """
    Симуляция торговли на тестовых данных.
    Будем покупать акции, если модель предсказывает рост цены, и продавать, если предсказывает падение
    :param model: Обученная модель
    :param X_test: Тестовые данные (временные окна)
    :param y_test: Реальные цены
    :param initial_capital: Начальный капитал
    :param commission: Комиссия за сделку (например, 0.1%)
    :return: История капитала, список сделок
    """
    # Получаем прогнозы модели
    predictions = model.predict(X_test)

    # Инициализация переменных
    capital = initial_capital
    position = 0  # Текущая позиция (0 - нет позиции, 1 - куплено, -1 - продано)
    trades = []  # Список для хранения сделок
    capital_history = []  # История изменения капитала

    for i in range(len(predictions) - 1):
        current_price = y_test[i]
        next_price = y_test[i + 1]
        predicted_change = predictions[i + 1] - predictions[i]

        # Сигнал на покупку (предсказание роста)
        if predicted_change > 0 and position <= 0:
            if position == -1:
                # Закрываем короткую позицию (покупаем)
                capital += position * current_price * (1 - commission)
                position = 0
            # Покупаем
            shares_to_buy = capital // (current_price * (1 + commission))
            if shares_to_buy > 0:
                capital -= shares_to_buy * current_price * (1 + commission)
                position = 1
                trades.append(('buy', current_price, shares_to_buy))

        # Сигнал на продажу (предсказание падения)
        elif predicted_change < 0 and position >= 0:
            if position == 1:
                # Закрываем длинную позицию (продаем)
                capital += position * current_price * (1 - commission)
                position = 0
            # Продаем (шорт)
            shares_to_sell = capital // (current_price * (1 + commission))
            if shares_to_sell > 0:
                capital += shares_to_sell * current_price * (1 - commission)
                position = -1
                trades.append(('sell', current_price, shares_to_sell))

        # Обновляем историю капитала
        if position == 1:
            capital_history.append(capital + (shares_to_buy * next_price))
        elif position == -1:
            capital_history.append(capital - (shares_to_sell * next_price))
        else:
            capital_history.append(capital)

    # Закрываем последнюю позицию, если она открыта
    if position == 1:
        capital += position * y_test[-1] * (1 - commission)
    elif position == -1:
        capital += position * y_test[-1] * (1 - commission)

    return capital_history, trades

In [354]:
# Симуляция торгов
capital_history, trades = simulate_trading(model, X_test, y_test, initial_capital=10000, commission=0.001)
# Вывод результатов
print(f"Конечный капитал: {capital_history[-1]:.2f}")
print(f"Количество сделок: {len(trades)}")
print(f"Первые 5 сделок: {trades[:5]}")

70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [356]:
# Добавление графика капитала в дашборд
def create_dashboard2(data, predictions, y_test, capital_history):
    st.title('Дашборд торговой стратегии')
    st.write("### График прогнозов и реальных цен")
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(y_test, label='Реальные цены')
    ax.plot(predictions, label='Прогнозы')
    ax.legend()
    st.pyplot(fig)

    st.write("### График изменения капитала")
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(capital_history, label='Капитал')
    ax.set_xlabel('Время')
    ax.set_ylabel('Капитал')
    ax.legend()
    st.pyplot(fig)

    st.write("### Метрики модели")
    metrics_df = pd.read_csv('metrics.csv')
    st.table(metrics_df)

In [358]:
create_dashboard2(data, predictions, y_test, capital_history)

2025-02-22 17:26:21.852 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 17:26:21.854 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 17:26:21.855 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 17:26:21.859 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 17:26:21.860 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 17:26:21.861 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 17:26:21.887 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-22 17:26:22.576 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar